In [14]:
from google.colab import files
uploaded = files.upload()

Saving tiktok_data2.csv to tiktok_data2.csv


In [2]:
!pip install pandas scikit-learn nltk --quiet
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import string
import re
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
data = pd.read_csv('tiktok_data.csv')
data

,id_video,duration_seconds,text_part,hashtags,human_time,author_followers,author_likes,id_sound,views,likes
0,1,13.0,А че такое?,"попугай, жако, попугайжако, говорящийпопугай, ...",2018-08-08 20:58:31,675900.0,7500000.0,7.028884e+18,144600000.0,4400000.0
1,2,9.0,Таксист в шоке😱 Inst: Andrey_Pryahin,NaN,2016-01-29 06:16:13,2300000.0,42600000.0,6.870179e+18,113300000.0,5300000.0
2,3,52.0,NaN,рекомендации,2018-12-30 09:24:42,58400.0,613400.0,7.052607e+18,105100000.0,505300.0
3,4,60.0,"Учитесь хитрить, как она 😄😉",NaN,2021-03-10 19:57:42,523900.0,11200000.0,6.984582e+18,96700000.0,3000000.0
4,5,16.0,телефонный пранк от говорящего ворона Карлушы,"пранк, шутка, говорящийворонкарлуша, воронгово...",2020-03-21 11:59:19,943700.0,16800000.0,7.052249e+18,139200000.0,5900000.0
...,...,...,...,...,...,...,...,...,...,...
755,756,0.0,❤️❤️,"foryoupage, ffyp",2025-06-01 14:38:06,1222.0,22300.0,7.511578e+18,219700.0,3180.0
756,757,58.0,BREAUNNA's short video with ♬ original sound,NaN,2023-10-28 05:16:45,14100.0,16300000.0,7.445616e+18,696800.0,130000.0
757,758,40.0,NaN,CapCut,2024-05-30 23:40:02,427800.0,1900000.0,7.491638e+18,177000.0,3991.0
758,759,5.0,Me in London 😅 anyone else? 😉 Maybe because I ...,"travel, traveltok, travelhumor",2022-10-06 15:48:17,3676.0,583600.0,7.407876e+18,422500.0,28700.0


In [11]:
from collections import defaultdict

class SimpleWord2Vec:
    def __init__(self, vector_size=50, window=5, min_count=2):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.word_vectors = {}

    def fit(self, sentences):
        word_freq = defaultdict(int)
        for sentence in sentences:
            for word in sentence:
                word_freq[word] += 1

        vocab = {word for word, count in word_freq.items() if count >= self.min_count}

        rng = np.random.RandomState(42)
        self.word_vectors = {
            word: rng.normal(scale=0.1, size=self.vector_size)
            for word in vocab
        }

        return self

    def __getitem__(self, word):
        return self.word_vectors.get(word, np.zeros(self.vector_size))

    def get_avg_vector(self, words):
        vectors = [self[word] for word in words if word in self.word_vectors]
        if vectors:
            return np.mean(vectors, axis=0)
        return np.zeros(self.vector_size)

In [5]:
class DataPreprocessor:
    def __init__(self):
        self.url_pattern = re.compile(r'https?://\S+|www\.\S+')
        self.emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)

    def clean_text(self, text):
        if pd.isna(text):
            return "empty_text"

        text = str(text).lower()
        text = self.url_pattern.sub('', text)
        text = self.emoji_pattern.sub('', text)
        text = re.sub(r'[^\w\s#]', ' ', text)
        tokens = word_tokenize(text)
        return ' '.join(tokens)

        if not tokens:
            return "empty_text"

        return ' '.join(tokens)

    def preprocess(self, data):
        data = data.dropna(how='all')
        data.drop('id_video', axis=1, inplace=True)
        data['duration_seconds'] = data['duration_seconds'] + 1
        data['cleaned_text'] = data['text_part'].apply(self.clean_text)
        data['cleaned_hashtags'] = data['hashtags'].apply(self.clean_text)
        data = data.dropna(subset=['cleaned_text', 'cleaned_hashtags'], how='all')
        data['text_length'] = data['text_part'].str.len().fillna(0)
        data['hashtags_count'] = data['hashtags'].str.count(' ').fillna(0) + 1
        numeric_cols = ['duration_seconds', 'author_followers', 'author_likes', 'views']
        for col in numeric_cols:
            data[col] = data[col].astype(str).str.replace(' ', '').str.replace(',', '').astype(float)
        data['human_time'] = pd.to_datetime(data['human_time'], errors='coerce')
        self.min_date = data['human_time'].min()
        data['year'] = data['human_time'].dt.year
        data['month'] = data['human_time'].dt.month
        data['day'] = data['human_time'].dt.day
        data['hour_of_day'] = data['human_time'].dt.hour
        data['min_of_hour'] = data['human_time'].dt.minute
        data['sec_of_min'] = data['human_time'].dt.second
        required_cols = numeric_cols + ['likes']
        data = data.dropna(subset=required_cols)
        data['id_sound'] = data['id_sound'].astype('int64')

        return data

class TextToEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=50, window=5, min_count=2):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None

    def fit(self, X, y=None):
        all_texts = X['cleaned_text'].apply(str.split).tolist() + \
                   X['cleaned_hashtags'].apply(str.split).tolist()

        self.model = SimpleWord2Vec(
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count
        )
        self.model.fit(all_texts)
        return self

    def transform(self, X):
        text_vectors = np.array([
            self.model.get_avg_vector(text.split())
            for text in X['cleaned_text']
        ])

        hashtag_vectors = np.array([
            self.model.get_avg_vector(hashtags.split())
            for hashtags in X['cleaned_hashtags']
        ])

        return np.hstack([text_vectors, hashtag_vectors])

In [6]:
preprocessor = DataPreprocessor()
processed_data = preprocessor.preprocess(data)

print(f"Исходное количество строк: {len(data)}")
print(f"Количество строк после очистки: {len(processed_data)}")
print(f"Удалено строк: {len(data) - len(processed_data)}")

Исходное количество строк: 760
Количество строк после очистки: 749
Удалено строк: 11


In [7]:
processed_data

,duration_seconds,text_part,hashtags,human_time,author_followers,author_likes,id_sound,views,likes,cleaned_text,cleaned_hashtags,text_length,hashtags_count,year,month,day,hour_of_day,min_of_hour,sec_of_min
0,14.0,А че такое?,"попугай, жако, попугайжако, говорящийпопугай, ...",2018-08-08 20:58:31,675900.0,7500000.0,7028884006213766144,144600000.0,4400000.0,а че такое,попугай жако попугайжако говорящийпопугай кира...,11.0,7.0,2018.0,8.0,8.0,20.0,58.0,31.0
1,10.0,Таксист в шоке😱 Inst: Andrey_Pryahin,NaN,2016-01-29 06:16:13,2300000.0,42600000.0,6870178609143860224,113300000.0,5300000.0,таксист в шоке inst andrey_pryahin,empty_text,36.0,1.0,2016.0,1.0,29.0,6.0,16.0,13.0
2,53.0,NaN,рекомендации,2018-12-30 09:24:42,58400.0,613400.0,7052606670136003584,105100000.0,505300.0,empty_text,рекомендации,0.0,1.0,2018.0,12.0,30.0,9.0,24.0,42.0
3,61.0,"Учитесь хитрить, как она 😄😉",NaN,2021-03-10 19:57:42,523900.0,11200000.0,6984581904523593728,96700000.0,3000000.0,учитесь хитрить как она,empty_text,27.0,1.0,2021.0,3.0,10.0,19.0,57.0,42.0
4,17.0,телефонный пранк от говорящего ворона Карлушы,"пранк, шутка, говорящийворонкарлуша, воронгово...",2020-03-21 11:59:19,943700.0,16800000.0,7052248674281327616,139200000.0,5900000.0,телефонный пранк от говорящего ворона карлушы,пранк шутка говорящийворонкарлуша воронговорит...,45.0,5.0,2020.0,3.0,21.0,11.0,59.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1.0,❤️❤️,"foryoupage, ffyp",2025-06-01 14:38:06,1222.0,22300.0,7511577560275274752,219700.0,3180.0,,foryoupage ffyp,4.0,2.0,2025.0,6.0,1.0,14.0,38.0,6.0
756,59.0,BREAUNNA's short video with ♬ original sound,NaN,2023-10-28 05:16:45,14100.0,16300000.0,7445615688498023424,696800.0,130000.0,breaunna s short video with original sound,empty_text,44.0,1.0,2023.0,10.0,28.0,5.0,16.0,45.0
757,41.0,NaN,CapCut,2024-05-30 23:40:02,427800.0,1900000.0,7491637932984634368,177000.0,3991.0,empty_text,capcut,0.0,1.0,2024.0,5.0,30.0,23.0,40.0,2.0
758,6.0,Me in London 😅 anyone else? 😉 Maybe because I ...,"travel, traveltok, travelhumor",2022-10-06 15:48:17,3676.0,583600.0,7407876181447543808,422500.0,28700.0,me in london anyone else maybe because i alrea...,travel traveltok travelhumor,81.0,3.0,2022.0,10.0,6.0,15.0,48.0,17.0


In [8]:
X = processed_data[['duration_seconds', 'cleaned_text', 'cleaned_hashtags',
                   'author_followers', 'author_likes', 'id_sound', 'views', 'year', 'month', 'day',
                   'hour_of_day', 'min_of_hour', 'sec_of_min', 'text_length', 'hashtags_count']]
y = (processed_data['likes'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
numeric_features = ['duration_seconds', 'author_followers', 'author_likes', 'id_sound', 'views', 'year', 'month', 'day',
                   'hour_of_day', 'min_of_hour', 'sec_of_min', 'text_length', 'hashtags_count']

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), numeric_features),
        ('text', TextToEmbeddingTransformer(vector_size=50), ['cleaned_text', 'cleaned_hashtags'])
        ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  ['duration_seconds',
                                                   'author_followers',
                                                   'author_likes', 'id_sound',
                                                   'views', 'year', 'month',
                                                   'day', 'hour_of_day',
                                                   'min_of_hour', 'sec_of_min',
                                                   'text_length',
                                                   'hashtags_count']),
                                                 ('text',
                                                  TextToEmbeddingTransformer(),
                                                  ['cleaned_text',
                                                   'cleaned_hashtags'])])),
                ('regressor', LinearRegression())])

In [13]:
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
medae = median_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("\nРезультаты модели:")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Root среднеквадратичная ошибка (RMSE): {rmse:.2f}")
print(f"Медиана абсолютных ошибок (MedAE): {medae:.2f}")
print(f"Коэффициент детерминации (R²): {r2:.4f}")


Результаты модели:
Среднеквадратичная ошибка (MSE): 1798253092449.41
Root среднеквадратичная ошибка (RMSE): 1340989.59
Медиана абсолютных ошибок (MedAE): 274075.04
Коэффициент детерминации (R²): 0.5537
